# Rule Based Classificiation

In [40]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import math

# --- Helper Functions ---
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return np.array(image)

def convolve(img, kernel):
    k = kernel.shape[0] // 2
    padded = np.pad(img, k, mode='edge')
    output = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i+2*k+1, j:j+2*k+1]
            output[i, j] = np.sum(region * kernel)
    return output

# ===============   PREPROCESSING AND FILTERING   ===============

def mean_filter(image, ksize=3):
    kernel = np.ones((ksize, ksize)) / (ksize * ksize)
    filtered = np.zeros_like(image)
    for c in range(3):
        filtered[:, :, c] = convolve(image[:, :, c], kernel)
    return filtered

def gaussian_filter(image, sigma=1):
    ksize = int(2 * np.ceil(3 * sigma) + 1)
    ax = np.linspace(-(ksize // 2), ksize // 2, ksize)
    gauss = np.exp(-0.5 * np.square(ax) / sigma**2)
    kernel = np.outer(gauss, gauss)
    kernel /= np.sum(kernel)
    if len(image.shape) == 2:
        filtered = convolve(image, kernel)
    else:
        filtered = np.zeros_like(image)
        for c in range(3):
            filtered[:, :, c] = convolve(image[:, :, c], kernel)
    return filtered

def median_filter(image, ksize=3):
    pad = ksize // 2
    h, w, c = image.shape
    padded = np.pad(image, ((pad, pad), (pad, pad), (0, 0)), mode='reflect')
    filtered = np.zeros_like(image)
    for i in range(h):
        for j in range(w):
            for ch in range(c):
                window = padded[i:i+ksize, j:j+ksize, ch]
                filtered[i, j, ch] = np.median(window)
    return filtered

def adaptive_median_filter(image, max_window_size=7):
    def process_channel(channel):
        padded = np.pad(channel, ((max_window_size//2, ), (max_window_size//2, )), mode='reflect')
        output = np.copy(channel)
        for i in range(channel.shape[0]):
            for j in range(channel.shape[1]):
                for k in range(3, max_window_size + 1, 2):
                    window = padded[i:i+k, j:j+k]
                    z_min = np.min(window)
                    z_max = np.max(window)
                    z_med = np.median(window)
                    z_xy = padded[i + max_window_size//2, j + max_window_size//2]
                    if z_min < z_med < z_max:
                        if z_min < z_xy < z_max:
                            output[i, j] = z_xy
                        else:
                            output[i, j] = z_med
                        break
                    elif k == max_window_size:
                        output[i, j] = z_med
        return output
    filtered = np.zeros_like(image)
    for c in range(3):
        filtered[:, :, c] = process_channel(image[:, :, c])
    return filtered

def unsharp_mask(image, sigma=1.0, amount=1.5):
    blurred = gaussian_filter(image, sigma)
    mask = image.astype(np.int32) - blurred.astype(np.int32)
    sharpened = image + amount * mask
    return np.clip(sharpened, 0, 255).astype(np.uint8)

def preprocess_image(img_np):
    img_np = mean_filter(img_np)
    img_np = gaussian_filter(img_np, sigma=1)
    img_np = median_filter(img_np)
    img_np = adaptive_median_filter(img_np)
    img_np = unsharp_mask(img_np, sigma=1.0, amount=1.5)
    return img_np

# ===============   COLOR SEGMENTATION   ===============
def rgb_to_hsv(rgb_img):
    rgb_norm = rgb_img.astype(np.float32) / 255.0
    r, g, b = rgb_norm[:, :, 0], rgb_norm[:, :, 1], rgb_norm[:, :, 2]
    v = np.max(rgb_norm, axis=2)
    min_rgb = np.min(rgb_norm, axis=2)
    delta = v - min_rgb
    s = np.where(v == 0, 0, delta / v)
    h = np.zeros_like(v)
    red_max = (v == r) & (delta > 0)
    green_max = (v == g) & (delta > 0)
    blue_max = (v == b) & (delta > 0)
    h[red_max] = ((g[red_max] - b[red_max]) / delta[red_max]) % 6
    h[green_max] = ((b[green_max] - r[green_max]) / delta[green_max]) + 2
    h[blue_max] = ((r[blue_max] - g[blue_max]) / delta[blue_max]) + 4
    h = (h * 60) / 2
    s = s * 255
    v = v * 255
    hsv_img = np.stack((h, s, v), axis=2).astype(np.uint8)
    return hsv_img

def segment_red_signs(hsv_img):
    lower_red1 = np.array([0, 0.3 * 255, 0.2 * 255], dtype=np.uint8)
    upper_red1 = np.array([15, 1.0 * 255, 1.0 * 255], dtype=np.uint8)
    lower_red2 = np.array([165, 0.3 * 255, 0.2 * 255], dtype=np.uint8)
    upper_red2 = np.array([180, 1.0 * 255, 1.0 * 255], dtype=np.uint8)
    mask1 = cv2.inRange(hsv_img, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_img, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)
    return red_mask

def segment_blue_signs(hsv_img):
    lower_blue = np.array([100, 0.3 * 255, 0.2 * 255], dtype=np.uint8)
    upper_blue = np.array([130, 1.0 * 255, 1.0 * 255], dtype=np.uint8)
    blue_mask = cv2.inRange(hsv_img, lower_blue, upper_blue)
    return blue_mask

def apply_morphological_operations(binary_mask, kernel_size=3):
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    eroded = cv2.erode(binary_mask, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=1)
    return dilated

def post_process_mask(binary_mask, kernel_size=3):
    morphed = apply_morphological_operations(binary_mask, kernel_size)
    return morphed

def segment_traffic_signs(image):
    hsv_img = rgb_to_hsv(image)
    red_mask = segment_red_signs(hsv_img)
    blue_mask = segment_blue_signs(hsv_img)
    red_mask_processed = post_process_mask(red_mask)
    blue_mask_processed = post_process_mask(blue_mask)
    combined_mask = np.bitwise_or(red_mask_processed, blue_mask_processed)
    return combined_mask

# ===============   EDGE DETECTION   ===============

def sobel_filters(img):
    Kx = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    Gx = convolve(img.astype(np.float32), Kx)
    Gy = convolve(img.astype(np.float32), Ky)
    magnitude = np.hypot(Gx, Gy).astype(np.uint8)
    angle = np.arctan2(Gy, Gx) * 180 / np.pi
    angle = (angle + 180) % 180
    return magnitude, angle

def non_maximum_suppression(magnitude, angle):
    Z = np.zeros(magnitude.shape)
    angle = angle // 45 * 45
    for i in range(1, magnitude.shape[0]-1):
        for j in range(1, magnitude.shape[1]-1):
            q = r = 255
            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                q = magnitude[i, j+1]
                r = magnitude[i, j-1]
            elif (22.5 <= angle[i, j] < 67.5):
                q = magnitude[i+1, j-1]
                r = magnitude[i-1, j+1]
            elif (67.5 <= angle[i, j] < 112.5):
                q = magnitude[i+1, j]
                r = magnitude[i-1, j]
            elif (112.5 <= angle[i, j] < 157.5):
                q = magnitude[i-1, j-1]
                r = magnitude[i+1, j+1]
            if (magnitude[i, j] >= q) and (magnitude[i, j] >= r):
                Z[i, j] = magnitude[i, j]
            else:
                Z[i, j] = 0
    return Z

def double_thresholding(img, low_ratio=0.05, high_ratio=0.15):
    high_threshold = img.max() * high_ratio
    low_threshold = high_threshold * low_ratio
    res = np.zeros(img.shape)
    weak = np.uint8(75)
    strong = np.uint8(255)
    strong_i, strong_j = np.where(img >= high_threshold)
    weak_i, weak_j = np.where((img <= high_threshold) & (img >= low_threshold))
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    return res, weak, strong

def edge_tracking(img, weak, strong=255):
    h, w = img.shape
    for i in range(1, h-1):
        for j in range(1, w-1):
            if img[i, j] == weak:
                if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                    or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                    or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                    img[i, j] = strong
                else:
                    img[i, j] = 0
    return img

def canny_edge(img):
    blurred = gaussian_filter(img)
    magnitude, angle = sobel_filters(blurred)
    non_max = non_maximum_suppression(magnitude, angle)
    thresholded, weak, strong = double_thresholding(non_max)
    final_edges = edge_tracking(thresholded, weak, strong)
    return final_edges

# =============== GEOMETRIC NORMALIZATION   ===============

def get_upright_rotation_angle(binary_mask):
    contours, _ = cv2.findContours(binary_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        rect = cv2.minAreaRect(contours[0])
        angle = rect[-1]
        if angle < -45:
            angle += 90
        return -angle
    return 0

def rotate_image(image, angle):
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REPLICATE)
    return rotated

def resize_image(img, target_size=(200, 200)):
    return cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR)

def geometric_normalization(binary_mask):
    angle = get_upright_rotation_angle(binary_mask)
    rotated_mask = rotate_image(binary_mask, angle)
    resized_mask = resize_image(rotated_mask, (200, 200))
    return resized_mask

# ===============   FEATURE EXTRACTION   ===============

def extract_corner_count(binary_mask):
    gray = (binary_mask * 255).astype(np.uint8)
    dst = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)
    corners = np.where(dst > 0.01 * dst.max())
    return len(corners[0])

def calculate_circularity(binary_mask):
    contours, _ = cv2.findContours(binary_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        area = cv2.contourArea(contours[0])
        perimeter = cv2.arcLength(contours[0], True)
        if perimeter == 0:
            return 0
        circularity = 4 * np.pi * area / (perimeter ** 2)
        return circularity
    return 0

def calculate_aspect_ratio_extent(binary_mask):
    contours, _ = cv2.findContours(binary_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        x, y, w, h = cv2.boundingRect(contours[0])
        area = cv2.contourArea(contours[0])
        if w == 0 or h == 0:
            return 0, 0
        aspect_ratio = w / h
        extent = area / (w * h)
        return aspect_ratio, extent
    return 0, 0

def calculate_average_hue(hsv_img, binary_mask):
    hue_values = hsv_img[:, :, 0][binary_mask > 0]
    if hue_values.size > 0:
        return np.mean(hue_values)
    return 0

def extract_features(normalized_mask, original_hsv_img, original_binary_mask):
    corner_count = extract_corner_count(normalized_mask)
    circularity = calculate_circularity(normalized_mask)
    aspect_ratio, extent = calculate_aspect_ratio_extent(normalized_mask)
    average_hue = calculate_average_hue(original_hsv_img, original_binary_mask)
    return {
        'corner_count': corner_count,
        'circularity': circularity,
        'aspect_ratio': aspect_ratio,
        'extent': extent,
        'average_hue': average_hue
    }

# =============== RULE BASED CLASSIFICATION ================

def classify_traffic_sign(features):
    """Applies rules to classify the traffic sign based on extracted features."""
    predicted_label = None
    if 0.2 < features['circularity'] < 0.8 and 0.5 < features['aspect_ratio'] < 1.6 and 0.5 < features['extent'] < 0.9 and 77 < features['average_hue'] < 102:
        predicted_label = 14
    elif 0 <= features['circularity']  < 0.9 and 0 <= features['average_hue'] < 83 and 0 <= features['aspect_ratio'] < 1.5 and 0 <= features['extent'] < 1.0:
        predicted_label = 1
    elif 0.3 < features['circularity'] < 0.8 and 9 <= features['average_hue'] < 32 and 0.6 < features['aspect_ratio'] < 1.1:
        predicted_label = 28

    # Rule for No Entry (Class ID 17): Circular with a central horizontal bar (might be captured by aspect ratio/extent) and red
    if features['circularity'] < 0.8 and 0 <= features['average_hue'] <= 136 and 0 <= features['aspect_ratio'] < 3.1 and 0 <= features['extent'] < 1.0:
        predicted_label = 17

    # Rule for Priority Road (Class ID 12): Diamond shape (lower circularity, specific aspect ratio/extent) and yellow/white
    if 0. <= features['circularity'] < 0.8 and 0 <= features['aspect_ratio'] < 2.2 and 0 <= features['extent'] < 1.0 and 0 <= features['average_hue'] < 113: # Assuming yellow hue range
        predicted_label = 12

    # Rule for Speed Limit 20 (Class ID 0): Generally circular and potentially red
    if 0.9 > features['circularity'] >= 0 and 31 < features['average_hue'] < 114 and 0.4 < features['extent'] < 1.0 and 0.3 <= features['aspect_ratio'] < 1.7:
        predicted_label = 0

    return predicted_label

# ===============   EVALUATION FUNCTIONS   ===============

def evaluate_performance(results_df):
    """Calculates overall accuracy and class-wise metrics."""
    correct_predictions = results_df['correct'].sum()
    total_predictions = len(results_df)
    overall_accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    class_metrics = {}
    for class_id in results_df['ground_truth'].unique():
        class_data = results_df[results_df['ground_truth'] == class_id]
        true_positives = class_data[class_data['ground_truth'] == class_data['predicted']].shape[0]
        false_positives = results_df[(results_df['ground_truth'] != class_id) & (results_df['predicted'] == class_id)].shape[0]
        false_negatives = results_df[(results_df['ground_truth'] == class_id) & (results_df['predicted'] != class_id)].shape[0]
        total_class = class_data.shape[0]
        class_accuracy = true_positives / total_class if total_class > 0 else 0
        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        class_metrics[class_id] = {'precision': precision, 'recall': recall, 'accuracy': class_accuracy}
    return overall_accuracy, class_metrics

def generate_confusion_matrix(results_df, class_labels):
    """Generates and saves the confusion matrix heatmap."""
    cm = pd.crosstab(results_df['ground_truth'], results_df['predicted'], rownames=['Actual'], colnames=['Predicted'])
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.title('Confusion Matrix')
    plt.savefig(CONFUSION_MATRIX_FILE)
    plt.close()

def write_metrics(overall_accuracy, class_metrics):
    """Writes the overall accuracy and class-wise metrics to a text file."""
    with open(METRICS_FILE, 'w') as f:
        f.write(f"Overall Accuracy: {overall_accuracy:.4f}\n\n")
        f.write("Class-wise Metrics:\n")
        for class_id, metrics in class_metrics.items():
            f.write(f"Class {class_id}:\n")
            f.write(f"  Precision: {metrics['precision']:.4f}\n")
            f.write(f"  Recall: {metrics['recall']:.4f}\n")
            f.write(f"  Accuracy: {metrics['accuracy']:.4f}\n")
            f.write("\n")

def write_results_csv(results_df):
    """Writes the results DataFrame to a CSV file."""
    results_df.to_csv(RESULTS_FILE, index=False)

# =============== MAIN EXECUTION ===============

if __name__ == "__main__":
    # --- Project Setup ---
    PROJECT_ROOT = "."
    CODE_DIR = os.path.join(PROJECT_ROOT, "code")
    RESULTS_FILE = os.path.join(PROJECT_ROOT, "results.csv")
    METRICS_FILE = os.path.join(PROJECT_ROOT, "metrics.txt")
    CONFUSION_MATRIX_FILE = os.path.join(PROJECT_ROOT, "confusion_matrix.png")
    DATA_DIR = os.path.join(PROJECT_ROOT, "./")
    os.makedirs(CODE_DIR, exist_ok=True)

    # 1. Select 6 to 8 different traffic sign classes
    selected_classes = [17,0, 1, 12, 14, 28]
    class_label_mapping = {0: 'speed limit 20', 1: 'speed limit 30', 28: 'school crossing', 12: 'priority road', 14: 'stop', 17: 'no entry'}
    selected_class_labels = [class_label_mapping[i] for i in selected_classes]

    results = []
    train_df = pd.read_csv("train.csv")
    train_subset_df = train_df[train_df['ClassId'].isin(selected_classes)].head(100 * len(selected_classes)).reset_index(drop=True)

    for index, row in train_subset_df.iterrows():
        image_path = os.path.join(DATA_DIR, row['Path'])
        ground_truth_label = row['ClassId']
        filename = os.path.basename(image_path)

        try:
            image = load_image(image_path)
            preprocessed_image = preprocess_image(image.copy())
            segmented_mask = segment_traffic_signs(preprocessed_image.copy())
            normalized_mask = geometric_normalization(segmented_mask.copy())
            hsv_image = rgb_to_hsv(preprocessed_image.copy())
            features = extract_features(normalized_mask.copy(), hsv_image.copy(), segmented_mask.copy())
            predicted_label = classify_traffic_sign(features)
            correct = 1 if predicted_label == ground_truth_label else 0
            results.append([filename, ground_truth_label, predicted_label, correct])
            print(f"Features: {features}")
            print(f"Processed: {filename} - Ground Truth: {ground_truth_label}, Predicted: {predicted_label}, Correct: {correct}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    results_df = pd.DataFrame(results, columns=['filename', 'ground_truth', 'predicted', 'correct'])
    overall_accuracy, class_metrics = evaluate_performance(results_df)
    generate_confusion_matrix(results_df, selected_class_labels)
    write_metrics(overall_accuracy, class_metrics)
    write_results_csv(results_df)
    print("Evaluation complete. Results, metrics, and confusion matrix saved.")

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 312)

# Random Forest

In [15]:
import os
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
from joblib import dump, load
import warnings
warnings.filterwarnings('ignore')

# --- Helper Functions ---
def load_image(image_path, target_size=(64, 64)):
    """Load and resize image to standard size to reduce memory usage"""
    try:
        image = Image.open(image_path).convert("RGB")
        image = image.resize(target_size)
        return np.array(image)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

def rgb_to_hsv(rgb_img):
    """Convert RGB to HSV using cv2 which is more efficient"""
    return cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)

def preprocess_batch(image_paths, target_size=(64, 64)):
    """Process a batch of images at once to improve efficiency"""
    processed_images = []
    valid_paths = []
    
    for path in image_paths:
        img = load_image(path, target_size)
        if img is not None:
            processed_images.append(img)
            valid_paths.append(path)
    
    return processed_images, valid_paths

# ===============   FEATURE EXTRACTION   ===============

def extract_features_batch(images):
    """Extract features from a batch of images"""
    features_list = []
    
    for img in images:
        # Convert to HSV for color features
        hsv_img = rgb_to_hsv(img)
        
        # Color features - histogram of hue and saturation channels
        h_hist = cv2.calcHist([hsv_img], [0], None, [16], [0, 180])
        s_hist = cv2.calcHist([hsv_img], [1], None, [16], [0, 256])
        v_hist = cv2.calcHist([hsv_img], [2], None, [16], [0, 256])
        
        # Shape features using HOG (Histogram of Oriented Gradients)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        # Calculate image statistics
        mean_color = np.mean(img, axis=(0, 1))
        std_color = np.std(img, axis=(0, 1))
        
        # Simple edge detection for shape features
        edges = cv2.Canny(gray, 100, 200)
        edge_density = np.sum(edges > 0) / (edges.shape[0] * edges.shape[1])
        
        # Basic shape descriptors
        try:
            # Convert grayscale to binary
            _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                # Get the largest contour
                largest_contour = max(contours, key=cv2.contourArea)
                area = cv2.contourArea(largest_contour)
                perimeter = cv2.arcLength(largest_contour, True)
                
                # Calculate circularity
                circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
                
                # Calculate aspect ratio using minimum area rectangle
                rect = cv2.minAreaRect(largest_contour)
                width, height = rect[1]
                aspect_ratio = width / height if height > 0 else 0
                if aspect_ratio < 1:
                    aspect_ratio = 1 / aspect_ratio
            else:
                circularity = 0
                aspect_ratio = 1
        except:
            circularity = 0
            aspect_ratio = 1
        
        # Collect all features
        feature_vector = np.concatenate([
            h_hist.flatten() / np.sum(h_hist) if np.sum(h_hist) > 0 else h_hist.flatten(),  # Normalized hue histogram
            s_hist.flatten() / np.sum(s_hist) if np.sum(s_hist) > 0 else s_hist.flatten(),  # Normalized saturation histogram
            v_hist.flatten() / np.sum(v_hist) if np.sum(v_hist) > 0 else v_hist.flatten(),  # Normalized value histogram
            mean_color,  # Mean color (3 values)
            std_color,   # Standard deviation of color (3 values)
            [edge_density, circularity, aspect_ratio]  # Shape features
        ])
        
        features_list.append(feature_vector)
    
    return np.array(features_list)

# =============== CLUSTERING FOR UNSUPERVISED LEARNING ===============

def perform_clustering(X, n_clusters=6):
    """Perform K-means clustering to group similar images"""
    print(f"Performing K-means clustering with {n_clusters} clusters...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X)
    
    # Calculate silhouette score if sklearn.metrics is available
    try:
        from sklearn.metrics import silhouette_score
        silhouette_avg = silhouette_score(X, cluster_labels)
        print(f"Silhouette Score: {silhouette_avg:.4f}")
    except:
        pass
    
    return kmeans, cluster_labels

# =============== TRAINING AND EVALUATION ===============

def train_models(X_train, y_train, X_val=None, y_val=None):
    """Train multiple models and select the best one"""
    models = {}
    best_score = 0
    best_model_name = None
    
    # 1. Random Forest Classifier
    print("Training Random Forest...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    rf_score = rf.score(X_val, y_val) if X_val is not None else rf.score(X_train, y_train)
    models["Random Forest"] = rf
    
    print(f"Random Forest validation score: {rf_score:.4f}")
    
    if rf_score > best_score:
        best_score = rf_score
        best_model_name = "Random Forest"
    
    # 2. SVM with RBF kernel (if dataset is not too large)
    if X_train.shape[0] < 5000:  # Only train SVM if dataset is manageable
        print("Training SVM...")
        svm = SVC(kernel='rbf', probability=True, random_state=42)
        svm.fit(X_train, y_train)
        svm_score = svm.score(X_val, y_val) if X_val is not None else svm.score(X_train, y_train)
        models["SVM"] = svm
        
        print(f"SVM validation score: {svm_score:.4f}")
        
        if svm_score > best_score:
            best_score = svm_score
            best_model_name = "SVM"
    
    print(f"Best model: {best_model_name} with score: {best_score:.4f}")
    return models, best_model_name

def evaluate_model(model, X_test, y_test, class_names):
    """Evaluate model performance and generate reports"""
    # Get predictions
    y_pred = model.predict(X_test)
    
    # Calculate overall accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    # Generate classification report
    report = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)
    
    # Generate confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    return accuracy, report, cm, y_pred

# =============== VISUALIZATION FUNCTIONS ===============

def plot_confusion_matrix(cm, class_names, output_file):
    """Plot and save confusion matrix"""
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()

def plot_feature_importance(model, feature_names, output_file):
    """Plot feature importance for tree-based models"""
    if hasattr(model, 'feature_importances_'):
        plt.figure(figsize=(12, 8))
        importances = model.feature_importances_
        indices = np.argsort(importances)[-20:]  # Get top 20 features
        
        plt.title('Feature Importances')
        plt.barh(range(len(indices)), importances[indices], align='center')
        plt.yticks(range(len(indices)), [feature_names[i] if i < len(feature_names) else f"Feature {i}" for i in indices])
        plt.xlabel('Relative Importance')
        plt.tight_layout()
        plt.savefig(output_file)
        plt.close()

def plot_clusters(X, cluster_labels, output_file, method='pca'):
    """Visualize clusters in 2D using PCA or t-SNE"""
    plt.figure(figsize=(10, 8))
    
    # Use PCA or t-SNE for dimensionality reduction
    if method == 'pca':
        from sklearn.decomposition import PCA
        reducer = PCA(n_components=2, random_state=42)
    else:  # t-SNE
        from sklearn.manifold import TSNE
        reducer = TSNE(n_components=2, random_state=42)
    
    # Reduce dimensions to 2D
    X_2d = reducer.fit_transform(X)
    
    # Plot clusters
    plt.scatter(X_2d[:, 0], X_2d[:, 1], c=cluster_labels, cmap='viridis', alpha=0.6)
    plt.title(f'Image Clusters Visualization using {method.upper()}')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.colorbar(label='Cluster')
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()

def plot_class_samples(images, labels, class_labels, output_dir, n_samples=5):
    """Save sample images from each class"""
    unique_classes = np.unique(labels)
    
    for class_id in unique_classes:
        # Get indices of images in this class
        class_indices = np.where(labels == class_id)[0]
        
        # Skip if no images in class
        if len(class_indices) == 0:
            continue
        
        # Select sample images (up to n_samples)
        sample_indices = class_indices[:min(n_samples, len(class_indices))]
        
        # Create a grid to display samples
        rows = 1
        cols = len(sample_indices)
        fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
        
        # Make axes iterable if there's only one sample
        if cols == 1:
            axes = [axes]
        
        # Plot each sample
        for i, idx in enumerate(sample_indices):
            axes[i].imshow(images[idx])
            axes[i].set_title(f'Class {class_labels[class_id]}')
            axes[i].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'class_{class_labels[class_id]}_samples.png'))
        plt.close()

def write_reports(accuracy, classification_report_dict, output_file):
    """Write detailed performance reports to a file"""
    with open(output_file, 'w') as f:
        f.write(f"Overall Accuracy: {accuracy:.4f}\n\n")
        f.write("Classification Report:\n")
        
        # Write per-class metrics
        for class_name, metrics in classification_report_dict.items():
            if class_name in ['accuracy', 'macro avg', 'weighted avg']:
                continue
                
            f.write(f"\nClass: {class_name}\n")
            f.write(f"  Precision: {metrics['precision']:.4f}\n")
            f.write(f"  Recall: {metrics['recall']:.4f}\n")
            f.write(f"  F1-score: {metrics['f1-score']:.4f}\n")
            f.write(f"  Support: {metrics['support']}\n")
        
        # Write summary metrics
        f.write("\nSummary:\n")
        for avg_type in ['macro avg', 'weighted avg']:
            if avg_type in classification_report_dict:
                f.write(f"{avg_type}:\n")
                metrics = classification_report_dict[avg_type]
                f.write(f"  Precision: {metrics['precision']:.4f}\n")
                f.write(f"  Recall: {metrics['recall']:.4f}\n")
                f.write(f"  F1-score: {metrics['f1-score']:.4f}\n")
                f.write(f"  Support: {metrics['support']}\n")

# =============== MAIN EXECUTION ===============

def main():
    # --- Project Setup ---
    PROJECT_ROOT = "."
    RESULTS_DIR = os.path.join(PROJECT_ROOT, "results")
    MODELS_DIR = os.path.join(PROJECT_ROOT, "models")
    TRAIN_DIR = os.path.join(PROJECT_ROOT, "Train")  # Update this to your Train directory
    
    # Create necessary directories
    os.makedirs(RESULTS_DIR, exist_ok=True)
    os.makedirs(MODELS_DIR, exist_ok=True)
    os.makedirs(os.path.join(RESULTS_DIR, "classes"), exist_ok=True)
    
    # Define output files
    CONFUSION_MATRIX_FILE = os.path.join(RESULTS_DIR, "confusion_matrix.png")
    FEATURE_IMPORTANCE_FILE = os.path.join(RESULTS_DIR, "feature_importance.png")
    METRICS_FILE = os.path.join(RESULTS_DIR, "metrics.txt")
    MODEL_FILE = os.path.join(MODELS_DIR, "best_model.joblib")
    
    print("Finding image files...")
    
    # Scan for class folders in the Train directory
    if not os.path.exists(TRAIN_DIR):
        print(f"Training directory {TRAIN_DIR} does not exist. Please check the path.")
        return
    
    class_folders = [folder for folder in os.listdir(TRAIN_DIR) 
                     if os.path.isdir(os.path.join(TRAIN_DIR, folder))]
    
    if not class_folders:
        print(f"No class folders found in {TRAIN_DIR}. Please check the directory structure.")
        return
    
    # Sort class folders and create class mapping
    class_folders.sort()  # Ensure consistent ordering
    class_to_index = {class_name: i for i, class_name in enumerate(class_folders)}
    index_to_class = {i: class_name for class_name, i in class_to_index.items()}
    
    print(f"Found {len(class_folders)} classes: {', '.join(class_folders)}")
    
    # Collect image paths and labels
    supported_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    image_files = []
    labels = []
    
    for class_name in class_folders:
        class_dir = os.path.join(TRAIN_DIR, class_name)
        class_idx = class_to_index[class_name]
        
        for file in os.listdir(class_dir):
            ext = os.path.splitext(file)[1].lower()
            if ext in supported_extensions:
                image_files.append(os.path.join(class_dir, file))
                labels.append(class_idx)
    
    print(f"Found {len(image_files)} image files across {len(class_folders)} classes")
    
    if len(image_files) == 0:
        print(f"No image files found in {TRAIN_DIR}. Please check the directory structure.")
        return
    
    # Process images in batches to avoid memory issues
    print("Processing images and extracting features...")
    batch_size = 50
    all_features = []
    all_images = []
    valid_paths = []
    valid_labels = []
    
    for i in range(0, len(image_files), batch_size):
        batch_paths = image_files[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        
        # Process batch
        processed_images, valid_batch_paths = preprocess_batch(batch_paths)
        
        if processed_images:
            # Extract features
            batch_features = extract_features_batch(processed_images)
            
            all_features.append(batch_features)
            all_images.extend(processed_images)
            valid_paths.extend(valid_batch_paths)
            
            # Keep only the labels for valid images
            for j, path in enumerate(batch_paths):
                if path in valid_batch_paths:
                    valid_labels.append(batch_labels[j])
            
        print(f"Processed batch {i//batch_size + 1}/{len(image_files)//batch_size + 1}")
    
    # Combine all batches
    X = np.vstack(all_features) if all_features else np.array([])
    y = np.array(valid_labels)
    
    if len(X) == 0:
        print("Error: No valid images processed. Please check your image paths and data.")
        return
    
    print(f"Final dataset: {X.shape[0]} samples with {X.shape[1]} features each")
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Create feature names for visualization
    feature_names = (
        [f"hue_bin_{i}" for i in range(16)] +
        [f"sat_bin_{i}" for i in range(16)] +
        [f"val_bin_{i}" for i in range(16)] +
        ["mean_r", "mean_g", "mean_b"] + 
        ["std_r", "std_g", "std_b"] +
        ["edge_density", "circularity", "aspect_ratio"]
    )
    
    # Split data for supervised learning
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Train models
    print("Training models using class labels...")
    models, best_model_name = train_models(X_train, y_train, X_test, y_test)
    best_model = models[best_model_name]
    
    # Save the best model
    dump(best_model, MODEL_FILE)
    print(f"Best model ({best_model_name}) saved to {MODEL_FILE}")
    
    # Evaluate multi-class model
    print("Evaluating model...")
    class_names = [index_to_class[i] for i in range(len(class_folders))]
    accuracy, report, cm, y_pred = evaluate_model(best_model, X_test, y_test, class_names)
    
    # Generate visualizations and reports
    print("Generating reports and visualizations...")
    plot_confusion_matrix(cm, class_names, CONFUSION_MATRIX_FILE)
    if hasattr(best_model, 'feature_importances_'):
        plot_feature_importance(best_model, feature_names, FEATURE_IMPORTANCE_FILE)
    write_reports(accuracy, report, METRICS_FILE)
    
    # Visualize sample images from each class
    plot_class_samples(all_images, y, index_to_class, os.path.join(RESULTS_DIR, "classes"))
    
    print(f"Evaluation complete. Results saved to {RESULTS_DIR}")
    print(f"Overall accuracy: {accuracy:.4f}")
    
    # Print class distribution
    print("\nClass distribution:")
    for class_idx, class_name in index_to_class.items():
        count = np.sum(y == class_idx)
        percentage = (count / len(y)) * 100
        print(f"Class {class_name}: {count} images ({percentage:.2f}%)")

if __name__ == "__main__":
    main()

Finding image files...
Found 43 classes: 0, 1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4, 40, 41, 42, 5, 6, 7, 8, 9
Found 39209 image files across 43 classes
Processing images and extracting features...
Processed batch 1/785
Processed batch 2/785
Processed batch 3/785
Processed batch 4/785
Processed batch 5/785
Processed batch 6/785
Processed batch 7/785
Processed batch 8/785
Processed batch 9/785
Processed batch 10/785
Processed batch 11/785
Processed batch 12/785
Processed batch 13/785
Processed batch 14/785
Processed batch 15/785
Processed batch 16/785
Processed batch 17/785
Processed batch 18/785
Processed batch 19/785
Processed batch 20/785
Processed batch 21/785
Processed batch 22/785
Processed batch 23/785
Processed batch 24/785
Processed batch 25/785
Processed batch 26/785
Processed batch 27/785
Processed batch 28/785
Processed batch 29/785
Processed batch 30/785
Processed batch 31/785
Process

Processed batch 335/785
Processed batch 336/785
Processed batch 337/785
Processed batch 338/785
Processed batch 339/785
Processed batch 340/785
Processed batch 341/785
Processed batch 342/785
Processed batch 343/785
Processed batch 344/785
Processed batch 345/785
Processed batch 346/785
Processed batch 347/785
Processed batch 348/785
Processed batch 349/785
Processed batch 350/785
Processed batch 351/785
Processed batch 352/785
Processed batch 353/785
Processed batch 354/785
Processed batch 355/785
Processed batch 356/785
Processed batch 357/785
Processed batch 358/785
Processed batch 359/785
Processed batch 360/785
Processed batch 361/785
Processed batch 362/785
Processed batch 363/785
Processed batch 364/785
Processed batch 365/785
Processed batch 366/785
Processed batch 367/785
Processed batch 368/785
Processed batch 369/785
Processed batch 370/785
Processed batch 371/785
Processed batch 372/785
Processed batch 373/785
Processed batch 374/785
Processed batch 375/785
Processed batch 

Processed batch 677/785
Processed batch 678/785
Processed batch 679/785
Processed batch 680/785
Processed batch 681/785
Processed batch 682/785
Processed batch 683/785
Processed batch 684/785
Processed batch 685/785
Processed batch 686/785
Processed batch 687/785
Processed batch 688/785
Processed batch 689/785
Processed batch 690/785
Processed batch 691/785
Processed batch 692/785
Processed batch 693/785
Processed batch 694/785
Processed batch 695/785
Processed batch 696/785
Processed batch 697/785
Processed batch 698/785
Processed batch 699/785
Processed batch 700/785
Processed batch 701/785
Processed batch 702/785
Processed batch 703/785
Processed batch 704/785
Processed batch 705/785
Processed batch 706/785
Processed batch 707/785
Processed batch 708/785
Processed batch 709/785
Processed batch 710/785
Processed batch 711/785
Processed batch 712/785
Processed batch 713/785
Processed batch 714/785
Processed batch 715/785
Processed batch 716/785
Processed batch 717/785
Processed batch 